previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [78]:
prev = pd.read_csv('./data/rawdata/previous_application.csv.zip',compression='zip')

prev = prev.sort_values(['SK_ID_CURR','SK_ID_PREV'], ascending = [True,False])

prev['cnt'] = 1

In [79]:
prev.shape
prev.columns
prev.head(50)

(1670214, 38)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'cnt'],
      dtype='object')

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,cnt
201668,1369693,100001,Consumer loans,3951.000,24835.50,23787.0,2520.00,24835.50,FRIDAY,13,...,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0,1
892077,1038818,100002,Consumer loans,9251.775,179055.00,179055.0,0.00,179055.00,SATURDAY,9,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,1
1021650,2636178,100003,Consumer loans,64567.665,337500.00,348637.5,0.00,337500.00,SUNDAY,17,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,1
1223745,2396755,100003,Consumer loans,6737.310,68809.50,68053.5,6885.00,68809.50,SATURDAY,15,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,1
575941,1810518,100003,Cash loans,98356.995,900000.00,1035882.0,NaN,900000.00,FRIDAY,12,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,1
935548,1564014,100004,Consumer loans,5357.250,24282.00,20106.0,4860.00,24282.00,FRIDAY,5,...,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,1
1378978,2495675,100005,Consumer loans,4813.200,44617.50,40153.5,4464.00,44617.50,THURSDAY,11,...,12.0,high,POS mobile with interest,365243.0,-706.0,-376.0,-466.0,-460.0,0.0,1
1259112,1857999,100005,Cash loans,NaN,0.00,0.0,NaN,NaN,FRIDAY,10,...,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN,1
617859,2827850,100006,Revolving loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,...,NaN,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN,1
1595430,2545789,100006,Cash loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,...,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN,1


In [53]:

prev_features = pd.concat([
    prev.SK_ID_PREV.groupby(prev.SK_ID_CURR).agg('count'),
    prev.NAME_CONTRACT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_CONTRACT_TYPE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_CONTRACT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.AMT_ANNUITY.groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
    prev.AMT_APPLICATION.groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
    prev.AMT_CREDIT.groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
    prev.AMT_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
    
    prev.AMT_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg(lambda x: sum(x>0)),
    prev.AMT_GOODS_PRICE.groupby(prev.SK_ID_CURR).agg(['sum','mean','max','min']),
    
    prev.WEEKDAY_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.WEEKDAY_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).nunique(),
    prev.WEEKDAY_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.HOUR_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.HOUR_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).nunique(),
    prev.HOUR_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev.HOUR_APPR_PROCESS_START.groupby(prev.SK_ID_CURR).agg('median'),
    
    prev.FLAG_LAST_APPL_PER_CONTRACT.groupby(prev.SK_ID_CURR).agg('min'),
    prev.NFLAG_LAST_APPL_IN_DAY.groupby(prev.SK_ID_CURR).agg('min'),
    
    prev.RATE_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.RATE_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev.RATE_INTEREST_PRIMARY.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.RATE_INTEREST_PRIMARY.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev.RATE_INTEREST_PRIVILEGED.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.RATE_INTEREST_PRIVILEGED.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_CASH_LOAN_PURPOSE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_CASH_LOAN_PURPOSE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_CASH_LOAN_PURPOSE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_CONTRACT_STATUS.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_CONTRACT_STATUS.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_CONTRACT_STATUS.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_CONTRACT_STATUS','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_CONTRACT_STATUS',values='cnt',aggfunc='count')
    
    prev.DAYS_DECISION.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_DECISION.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_PAYMENT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_PAYMENT_TYPE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_PAYMENT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_PAYMENT_TYPE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_PAYMENT_TYPE',values='cnt',aggfunc='count')
    
    prev.CODE_REJECT_REASON.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.CODE_REJECT_REASON.groupby(prev.SK_ID_CURR).nunique(),
    prev.CODE_REJECT_REASON.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['CODE_REJECT_REASON','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='CODE_REJECT_REASON',values='cnt',aggfunc='count')
    
    prev.NAME_TYPE_SUITE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_TYPE_SUITE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_TYPE_SUITE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_TYPE_SUITE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_TYPE_SUITE',values='cnt',aggfunc='count')
    
    prev.NAME_CLIENT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_CLIENT_TYPE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_CLIENT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_CLIENT_TYPE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_CLIENT_TYPE',values='cnt',aggfunc='count')
    
    prev.NAME_GOODS_CATEGORY.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_GOODS_CATEGORY.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_GOODS_CATEGORY.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_PORTFOLIO.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_PORTFOLIO.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_PORTFOLIO.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_PORTFOLIO','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_PORTFOLIO',values='cnt',aggfunc='count')
    
    prev.NAME_PRODUCT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_PRODUCT_TYPE.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_PRODUCT_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_PRODUCT_TYPE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_PRODUCT_TYPE',values='cnt',aggfunc='count')
    
    prev.CHANNEL_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.CHANNEL_TYPE.groupby(prev.SK_ID_CURR).nunique(),
    prev.CHANNEL_TYPE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['CHANNEL_TYPE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='CHANNEL_TYPE',values='cnt',aggfunc='count')
    
    prev.SELLERPLACE_AREA.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.SELLERPLACE_AREA.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_SELLER_INDUSTRY.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_SELLER_INDUSTRY.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_SELLER_INDUSTRY.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_SELLER_INDUSTRY','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_SELLER_INDUSTRY',values='cnt',aggfunc='count')
    
    prev.CNT_PAYMENT.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.CNT_PAYMENT.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NAME_YIELD_GROUP.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.NAME_YIELD_GROUP.groupby(prev.SK_ID_CURR).nunique(),
    prev.NAME_YIELD_GROUP.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['NAME_YIELD_GROUP','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_YIELD_GROUP',values='cnt',aggfunc='count')
    
    prev.PRODUCT_COMBINATION.groupby(prev.SK_ID_CURR).agg(lambda x: ','.join(set(x))),
    prev.PRODUCT_COMBINATION.groupby(prev.SK_ID_CURR).nunique(),
    prev.PRODUCT_COMBINATION.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    prev[['PRODUCT_COMBINATION','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='PRODUCT_COMBINATION',values='cnt',aggfunc='count')
    
    prev.DAYS_FIRST_DRAWING.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_FIRST_DRAWING.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.DAYS_FIRST_DUE.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_FIRST_DUE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.DAYS_LAST_DUE_1ST_VERSION.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_LAST_DUE_1ST_VERSION.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.DAYS_LAST_DUE.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_LAST_DUE.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.DAYS_TERMINATION.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']),
    prev.DAYS_TERMINATION.groupby(prev.SK_ID_CURR).agg(lambda x: list(x)[0]),
    
    prev.NFLAG_INSURED_ON_APPROVAL.groupby(prev.SK_ID_CURR).agg('min'),

    
], axis = 1)

 365243.0    934444
NaN          673065
-228.0          123
-212.0          121
-224.0          121
-223.0          119
-220.0          118
-210.0          117
-235.0          117
-240.0          116
-227.0          115
-226.0          115
-244.0          114
-217.0          112
-239.0          111
-255.0          110
-249.0          109
-197.0          109
-196.0          108
-263.0          108
-237.0          108
-202.0          108
-241.0          108
-266.0          107
-238.0          107
-252.0          106
-256.0          106
-259.0          106
-215.0          106
-230.0          106
              ...  
-1865.0           1
-1866.0           1
-1867.0           1
-1577.0           1
-1664.0           1
-1839.0           1
-1834.0           1
-1775.0           1
-1609.0           1
-1840.0           1
-1779.0           1
-1781.0           1
-1783.0           1
-1786.0           1
-1787.0           1
-1611.0           1
-1791.0           1
-1797.0           1
-1607.0           1


In [107]:
# prev[['SK_ID_PREV','SK_ID_CURR','NAME_CONTRACT_TYPE','FLAG_LAST_APPL_PER_CONTRACT','NAME_CONTRACT_STATUS','CODE_REJECT_REASON','NAME_TYPE_SUITE']].head(50)

# prev.PRODUCT_COMBINATION.value_counts()
# prev.RATE_DOWN_PAYMENT.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median'])
# prev[['NAME_PRODUCT_TYPE','SK_ID_CURR','cnt']].pivot_table(index = 'SK_ID_CURR',columns='NAME_PRODUCT_TYPE',values='cnt',aggfunc='count')
# prev.DAYS_FIRST_DUE.groupby(prev.SK_ID_CURR).agg(['mean','max','min','median']).astype(%.2f)

SyntaxError: invalid syntax (<ipython-input-107-077c5728a393>, line 6)

In [106]:
prev.DAYS_FIRST_DUE.astype(str)

201668      -1709.0
892077       -565.0
1021650      -797.0
1223745     -2310.0
575941       -716.0
935548       -784.0
1378978      -706.0
1259112         nan
617859          nan
1595430         nan
1333657      -545.0
830967       -281.0
98162        -151.0
1607443         nan
900957     365243.0
1232752         nan
1131133         nan
87569           nan
1260339     -1736.0
1061094     -2326.0
13392        -834.0
1483068     -1076.0
693308       -344.0
800095       -609.0
320227       -339.0
906992      -2299.0
600145          nan
604799      -2491.0
1286497       -43.0
946687       -795.0
             ...   
120009       -419.0
1003590     -2842.0
1387092    365243.0
345758      -1092.0
1168620     -2649.0
1239697      -710.0
234061          nan
478898      -1495.0
1440475         nan
1345759         nan
791939      -1038.0
490103       -327.0
1467403      -707.0
553648          nan
1094543         nan
935792       -276.0
292302       -210.0
444497      -2466.0
181558      -1866.0
